In [392]:
import pandas as pd 
import numpy as np

In [393]:
train_val_df_raw = pd.read_csv('/Users/thananpornsethjinda/Desktop/internship/projects/titanic-survival-prediction/data/train.csv')
test_df_raw = pd.read_csv('/Users/thananpornsethjinda/Desktop/internship/projects/titanic-survival-prediction/data/test.csv')

In [394]:
from basic_cleaning import clean_dataframe

train_val_df = clean_dataframe(train_val_df_raw)

print("-" * 100)

test_df = clean_dataframe(test_df_raw)

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age         177
cabin       687
embarked      2
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket', 'cabin']
[INFO] Potential numeric outliers detected:
{'age': np.int64(11), 'sibsp': np.int64(46), 'parch': np.int64(213), 'fare': np.int64(116)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.
----------------------------------------------------------------------------------------------------
[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age       86
fare       1
cabin    327
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket']
[INFO] Potential numeric outliers detected:
{'age': np.int64(2), 'sibsp': np.int64(11), 'parch': np.int64(94), 'fare': n

In [395]:
combined = pd.concat([train_val_df, test_df])

In [396]:
train_val_df

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"braund, mr. owen harris",male,22.0,1,0,a/5 21171,7.2500,NaN,s
1,2,1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,pc 17599,71.2833,c85,c
2,3,1,3,"heikkinen, miss. laina",female,26.0,0,0,ston/o2. 3101282,7.9250,NaN,s
3,4,1,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,113803,53.1000,c123,s
4,5,0,3,"allen, mr. william henry",male,35.0,0,0,373450,8.0500,NaN,s
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"montvila, rev. juozas",male,27.0,0,0,211536,13.0000,NaN,s
887,888,1,1,"graham, miss. margaret edith",female,19.0,0,0,112053,30.0000,b42,s
888,889,0,3,"johnston, miss. catherine helen ""carrie""",female,NaN,1,2,w./c. 6607,23.4500,NaN,s
889,890,1,1,"behr, mr. karl howell",male,26.0,0,0,111369,30.0000,c148,c


In [397]:
len(train_val_df['ticket'].unique())

681

In [398]:
train_val_df['ticket'].value_counts()

ticket
347082      7
ca. 2343    7
1601        7
3101295     6
ca 2144     6
           ..
9234        1
19988       1
2693        1
pc 17612    1
370376      1
Name: count, Length: 681, dtype: int64

In [399]:
train_val_df.loc[train_val_df['ticket'] == '347082']

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
13,14,0,3,"andersson, mr. anders johan",male,39.0,1,5,347082,31.275,NaN,s
119,120,0,3,"andersson, miss. ellis anna maria",female,2.0,4,2,347082,31.275,NaN,s
541,542,0,3,"andersson, miss. ingeborg constanzia",female,9.0,4,2,347082,31.275,NaN,s
542,543,0,3,"andersson, miss. sigrid elisabeth",female,11.0,4,2,347082,31.275,NaN,s
610,611,0,3,"andersson, mrs. anders johan (alfrida konstant...",female,39.0,1,5,347082,31.275,NaN,s
813,814,0,3,"andersson, miss. ebba iris alfrida",female,6.0,4,2,347082,31.275,NaN,s
850,851,0,3,"andersson, master. sigvard harald elias",male,4.0,4,2,347082,31.275,NaN,s


In [400]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

In [401]:
X_train, y_train = train_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']], train_df['survived']
X_val, y_val = val_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']], val_df['survived']
X_test = test_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']]

In [402]:
X_train.groupby('pclass')['age'].mean() 
# you could technically impute the average age for each class instead of imputing one mean for every missing value

pclass
1    37.891049
2    29.843972
3    25.162917
Name: age, dtype: float64

In [403]:
TitleDict = {"capt": "officer","col": "officer","major": "officer","jonkheer": "royalty", \
             "don": "royalty", "sir" : "royalty","dr": "royalty","rev": "royalty", \
             "countess":"royalty", "mme": "mrs", "mlle": "miss", "ms": "mrs","mr" : "mr", \
             "mrs" : "mrs","miss" : "miss","master" : "master","lady" : "royalty"}

def preprocessing(df): 

    df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)

    df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())

    df['ticket_per_person'] = df['fare']/df['ticket_people']

    df['title'] = df.title.map(TitleDict)

    df.loc[(df.title == 'miss') & (df.parch != 0) & (df.ticket_people > 1), 'title'] = "female_child"

    return df


In [404]:
X_train_2 = preprocessing(X_train)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)
/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/2284188230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())


In [405]:
X_val_2 = preprocessing(X_val)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)
/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/2284188230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())


In [406]:
X_test_2 = preprocessing(X_test)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)


In [407]:
X_test_2.at[414, 'title'] = 'royalty'

In [417]:
grouped = X_train_2.groupby(['pclass', 'sex', 'title'])['age'].mean()


/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_57814/532939052.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = X_train_2.groupby(['pclass', 'sex', 'title'])['age'].mean()


In [418]:
grouped

pclass  sex     title       
1       female  female_child    20.400000
                master                NaN
                miss            34.000000
                mr                    NaN
                mrs             40.033333
                officer               NaN
                royalty         40.500000
        male    female_child          NaN
                master           2.460000
                miss                  NaN
                mr              40.431818
                mrs                   NaN
                officer         56.600000
                royalty         42.000000
2       female  female_child    10.000000
                master                NaN
                miss            30.593750
                mr                    NaN
                mrs             33.031250
                officer               NaN
                royalty               NaN
        male    female_child          NaN
                master           2.437500
     

In [419]:
grouped = grouped.reset_index()[['sex','pclass','title','age']]

In [422]:
grouped = grouped.dropna()

In [423]:
grouped

,sex,pclass,title,age
0,female,1,female_child,20.400000
2,female,1,miss,34.000000
4,female,1,mrs,40.033333
6,female,1,royalty,40.500000
8,male,1,master,2.460000
10,male,1,mr,40.431818
12,male,1,officer,56.600000
13,male,1,royalty,42.000000
14,female,2,female_child,10.000000
16,female,2,miss,30.593750


In [425]:
def fill_age(x): 
    return grouped[(grouped.sex == x.sex) & (grouped.pclass == x.pclass) & (grouped.title == x.title)]['age'].values[0]


ideas for feature engineering: 
1. create family size 
2. children present? i dont think so because this column wont work out for children themsevles 
3. the cabin section (A,B,C,D,E and all that) - apparently more than 70% of the data is missing so imputing would just create more noise and possible inaccuracies 
4. extract the title and p-class, and group by title and pclass and find the mean for age 

how are we going find family size - we can just add SibSp and Parch 

eda on fare and survival rate 

eda on embarked and survival rate 